In [1]:
import os
import pandas as pd
import numpy as np
import json
import requests
import zipfile
import datetime
import re
from tqdm import tqdm

In [2]:
df = pd.read_csv(os.path.join("medicine_dataset.csv"))
print(len(df))

173528


In [4]:
subset = df.loc[df["brand_name"] == df["generic_name"]]
print(len(subset))
subset.to_csv("no_name_drugs.csv",index=False)

42452


In [10]:
drug_df = pd.read_csv(os.path.join("drug","drug_dataset.csv"))

generic = {}
generic_dup = {}
for i, row in tqdm(drug_df.iterrows()):
    try:
        gen = row["generic_name"].lower()
    except:
        continue
    if gen not in generic and gen not in generic_dup:
        generic[gen] = row["id"]
    else:
        if gen not in generic_dup:
            generic_dup[gen] = [generic[gen], row["id"]]
            generic.pop(gen,None)
        else:
            generic_dup[gen].append(row["id"])
print(len(generic_dup))

205302it [00:12, 16028.26it/s]


8684


In [11]:
print(len(generic))

8144


In [12]:
drug_df = pd.read_csv(os.path.join("drug","drug_dataset.csv"))

brand = {}
brand_dup = {}
for i, row in tqdm(drug_df.iterrows()):
    try:
        bra = row["brand_name"].lower()
    except:
        continue
    if bra not in brand and bra not in brand_dup:
        brand[bra] = row["id"]
    else:
        if bra not in brand_dup:
            brand_dup[bra] = [brand[bra], row["id"]]
            brand.pop(bra,None)
        else:
            brand_dup[bra].append(row["id"])
print(len(brand_dup))

205302it [00:12, 16242.00it/s]


17971


In [13]:
print(len(brand))

15338


In [18]:
drug_df = drug_df.fillna("")
drug_df = drug_df.loc[drug_df["generic_name"]!=""]
drug_df = drug_df.loc[drug_df["brand_name"]!=""]
drug_df["drug_name"] = drug_df["brand_name"].str.lower() + "---" + drug_df["generic_name"].str.lower()
print(len(drug_df))

178555


In [20]:
meds = {}
meds_dup = {}
for i, row in tqdm(drug_df.iterrows()):
    med = row["drug_name"]
    if med not in meds and med not in meds_dup:
        meds[med] = row["id"]
    else:
        if med not in meds_dup:
            meds_dup[med] = [meds[med], row["id"]]
            meds.pop(med,None)
        else:
            meds_dup[med].append(row["id"])
print(len(meds_dup))

178555it [00:11, 15949.43it/s]


22655


In [21]:
print(len(meds))

61586


In [25]:
count = 0
for key in meds_dup:
    print(key)
    count+=1
    if count == 10:
        break

acetaminophen and codeine phosphate---acetaminophen and codeine phosphate
hydrocodone bitartrate and acetaminophen---hydrocodone bitartrate and acetaminophen
kids relief---arnica montana
labetalol hydrochloride---labetalol hydrochloride
kids relief---arsenicum album
kids relief---ferrum phosphoricum
oxycodone hydrochloride---oxycodone hydrochloride
kids relief---drosera
kids relief---bryonia
kids relief---ipecacuanha


In [36]:
comp_df = pd.read_csv(os.path.join("drug","active_ingredient_dataset.csv"))
meds_df = drug_df[["id", "drug_name", "product_type", "manufacturer_name"]]

data = {"id":[], "components":[]}
vocab = {}
for i, row in tqdm(comp_df.iterrows()):
    subset = comp_df.loc[comp_df["id"] == row["id"]]
    comps = subset["active_ingredient"].values
    if row["id"] not in vocab:
        vocab[row["id"]] = len(comps)
        data["id"].append(row["id"])
        data["components"].append(", ".join(comps))

223302it [02:31, 1473.58it/s]


In [43]:
comp_df = pd.read_csv(os.path.join("drug","active_ingredient_dataset.csv"))
print(len(comp_df))
comp_df = pd.DataFrame.from_dict(data)
print(len(comp_df))

223302
124017


In [53]:
df = drug_df.join(comp_df.set_index('id'), on="id", how="inner")
print(len(df))
print(df.head())

173528
   id   brand_name                  generic_name  product_type  \
0   0    Strattera     Atomoxetine hydrochloride             1   
1   1  flucytosine                   flucytosine             1   
2   2    Theragold                Arnica Montana             0   
3   2    Theragold  Melissa Officinalis Seed Oil             0   
4   2    Theragold  Calendula Officinalis Flower             0   

                manufacturer_name                                 drug_name  \
0           Eli Lilly and Company     strattera---atomoxetine hydrochloride   
1  West-Ward Pharmaceuticals Corp                 flucytosine---flucytosine   
2     Square One New Zealand Llc.                theragold---arnica montana   
3     Square One New Zealand Llc.  theragold---melissa officinalis seed oil   
4     Square One New Zealand Llc.  theragold---calendula officinalis flower   

                                          components  
0                          ATOMOXETINE HYDROCHLORIDE  
1          

In [54]:
df["components"] = df["components"].str.lower()
df["brand_name"] = df["brand_name"].str.lower()
df["generic_name"] = df["generic_name"].str.lower()
df["brand_id"] = df["id"]
df["id"] = np.arange(len(df))
df = df[["id", "brand_id", "brand_name", "generic_name", "components", "product_type", "manufacturer_name"]]
df.to_csv("medicine_dataset.csv",index=False)